In [1]:
import yfinance as yf
from datetime import date

# Purpose

This notebook is where I am going to explore making my own transformers and pipelines

In [4]:
ticker = yf.Ticker('BTC')
df = ticker.history(start=date(2021, 5, 1), end=date(2021, 7, 1))

In [5]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2021-05-03,98.910004,98.945000,98.910004,98.944603,2307,0,0
2021-05-04,98.959999,99.050003,98.959999,98.985001,2847,0,0
2021-05-05,99.000000,99.059998,98.994003,99.059998,2950,0,0
2021-05-06,99.099998,99.100998,99.030998,99.074997,4313,0,0
2021-05-07,99.150002,99.214996,99.147499,99.147499,1834,0,0
2021-05-10,99.199997,99.199997,99.070000,99.110001,2968,0,0
2021-05-11,99.110001,99.110001,99.110001,99.110001,0,0,0
